<a href="https://colab.research.google.com/github/navas017/Grupo40-Practica1/blob/main/Practica1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importación de librerías** 

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
import math

# **PROCESAMIENTO DE DATOS**

Lectura de archivos

In [ ]:
# Aprendizaje
disp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/disp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

# Test
comp_df = pd.read_csv("/content/drive/MyDrive/Aprendizaje Automático/Datos/comp_st40ns1.txt.bz2",
 compression="bz2",
 index_col=0)

Partición de datos en conjuntos de entrenamiento y test

In [ ]:
train = disp_df.iloc[:5]
test = disp_df.iloc[3650:]

Randomización de las filas en los ficheros de train y test y valores de entrenamiento

In [ ]:
# Para la randomizacion de filas se usa la semilla 100429142
train_random = train.sample(frac=1, random_state=100429142).reset_index(drop=True)
test_random = test.sample(frac=1, random_state=100429142).reset_index(drop=True)

# Separa las variables independientes (X) de la variable dependiente (y)
X_train = train_random.iloc[:, :-1]
Y_train = train_random.iloc[:, -1]

# **MODELOS BÁSICOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo *KNN* con omisión de hiperparámetros

In [ ]:
# Crear el modelo con los parámetros por defecto
knn = KNeighborsRegressor()

# Entrena el modelo con el conjunto de entrenamiento
knn.fit(X_train, Y_train)

# Realiza predicciones en el conjunto de entrenamiento
y_pred_train = knn.predict(X_train)

# Calcula el MAE en el conjunto de entrenamiento
mae_train = mean_absolute_error(Y_train, y_pred_train)

# Calcula el RMSE en el conjunto de entrenamiento
rmse_train = math.sqrt(mean_squared_error(Y_train, y_pred_train))

# Muestra el valor del MAE y RMSE en el conjunto de entrenamiento
print('MAE en el conjunto de entrenamiento:', mae_train)
print('RMSE en el conjunto de entrenamiento:', rmse_train)

MAE en el conjunto de entrenamiento: 2320715.78630137
RMSE en el conjunto de entrenamiento: 3221546.9694545004


Modelo Árbol de decisión con omisión de hiperparámetros

In [ ]:
# Crear el modelo con los parámetros por defecto
tree = DecisionTreeRegressor()

# Entrenar el modelo
tree.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = tree.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Modelo Árbol de regresión lineal con omisión de hiperparámetros

In [ ]:
# Crear modelo
modelo_rl = LinearRegression()

# Entrenar modelo
modelo_rl.fit(X_train, Y_train)

# Predecir valores para datos de test
y_pred = modelo_rl.predict(X_train)

mae = mean_absolute_error(Y_train, y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, y_pred))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 2175824.6934650526
RMSE: 3137001.6072183605


## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo KNN con hiperparámetros óptimos

In [ ]:
knn = KNeighborsRegressor()

# Definir cuadrícula de valores de hiperparámetros a explorar
param_grid = {'n_neighbors': [1, 3, 5, 7, 9], 'p': [1, 2]}

# Definir métricas de evaluación a utilizar
scoring = {'MAE': make_scorer(mean_absolute_error), 'RMSE': make_scorer(mean_squared_error, squared=False)}

# Realizar búsqueda exhaustiva de los mejores valores de los hiperparámetros
grid_search = GridSearchCV(knn, param_grid=param_grid, scoring=scoring, refit='RMSE', cv=5)
grid_search.fit(X_train, Y_train)

# Imprimir mejores valores de los hiperparámetros y su rendimiento en términos de MAE y RMSE
print('Mejores valores de hiperparámetros:', grid_search.best_params_)
print('MAE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_MAE'][grid_search.best_index_])
print('RMSE con hiperparámetros optimizados:', grid_search.cv_results_['mean_test_RMSE'][grid_search.best_index_])

Modelo Árbol de Decisión con hiperparámetros óptimos

In [16]:
# Definir el modelo
tree = DecisionTreeRegressor()

# Definir los parámetros a probar
param_grid = {'max_depth': [3, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 8]}

# Definir la búsqueda por validación cruzada
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='neg_mean_squared_error')

# Entrenar el modelo con los datos de entrenamiento
grid_search.fit(X_train, Y_train)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = best_model.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("Mejores parámetros: ", best_params)
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Mejores parámetros:  {'max_depth': 7, 'min_samples_leaf': 2}
MAE: 371280.00
RMSE: 421678.49


Modelo Árbol de Regresión Lineal con hiperparámetros óptimos

# **MODELOS AVANZADOS**

## **OMISIÓN DE HIPERPARÁMETROS**

Modelo SVMs con omisión de hiperparámetros

In [19]:
# Crear un clasificador SVM sin ajuste de hiperparámetros
clf = SVC(kernel='linear', C=1, random_state=0)

# Entrenar el clasificador en los datos de entrenamiento
clf.fit(X_train, Y_train)

# Hacer predicciones en los datos de entrenamiento
y_pred_train = clf.predict(X_train)

# Evaluar el rendimiento del clasificador en los datos de entrenamiento
accuracy_train = clf.score(X_train, Y_train)
print("Accuracy en los datos de entrenamiento:", accuracy_train)

Accuracy en los datos de entrenamiento: 1.0


Modelo Random Forests con omisión de hiperparámetros

In [21]:
# Crear modelo con los parámetros por defecto
rf = RandomForestRegressor()

# Entrenar el modelo
rf.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = rf.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

MAE: 381391.20
RMSE: 483248.84


## **OPTIMIZACIÓN DE HIPERPARÁMETROS**

Modelo SVMs con hiperparámetros óptimos

In [18]:
# Definir los parámetros a optimizar
param_grid = {'C': [0.1, 1, 10], 'epsilon': [0.1, 0.2, 0.3], 'gamma': ['scale', 'auto']}

# Crear el modelo SVR
svm = SVR()

# Realizar la búsqueda de los mejores parámetros
svm_opt = GridSearchCV(svm, param_grid, scoring='neg_mean_absolute_error', cv=5)
svm_opt.fit(X_train, Y_train)

# Obtener los mejores parámetros
best_params = svm_opt.best_params_
print("Mejores parámetros:", best_params)

# Entrenar el modelo con los mejores parámetros
svm = SVR(**best_params)
svm.fit(X_train, Y_train)

# Realizar predicciones en el conjunto de entrenamiento
Y_pred = svm.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Mejores parámetros: {'C': 10, 'epsilon': 0.1, 'gamma': 'scale'}
MAE: 1184580.00
RMSE: 1720863.07


Modelo Random Forests con hiperparámetros óptimos

In [22]:
# Crear modelo de Random Forest
rf = RandomForestRegressor()

# Definir grid de hiperparámetros a evaluar
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Crear objeto GridSearchCV para encontrar los mejores parámetros
grid_rf = GridSearchCV(rf, param_grid, cv=3, n_jobs=-1, verbose=1)

# Entrenar modelo con parámetros optimizados
grid_rf.fit(X_train, Y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros: ", grid_rf.best_params_)

# Predecir valores para datos de test usando el modelo optimizado
Y_pred = grid_rf.predict(X_train)

# Calcular el MAE y el RMSE
mae = mean_absolute_error(Y_train, Y_pred)
rmse = math.sqrt(mean_squared_error(Y_train, Y_pred))

# Imprimir los resultados
print("MAE: {:.2f}".format(mae))
print("RMSE: {:.2f}".format(rmse))

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Mejores parámetros:  {'max_depth': None, 'max_features': 'auto', 'n_estimators': 50}
MAE: 405273.60
RMSE: 456853.60


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
